In [4]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 
tf.disable_eager_execution() 

In [24]:
data = pd.read_csv('news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [25]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [26]:
# These are some variables required for the model training.
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [27]:
# Tokenization 
# This process divides a large piece of continuous text into distinct units or tokens basically. 
# Here we use columns separately for a temporal basis as a pipeline just for good accuracy.
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

In [28]:
tokenizer1 = Tokenizer() 
tokenizer1.fit_on_texts(title) 
word_index1 = tokenizer1.word_index 
vocab_size1 = len(word_index1) 
sequences1 = tokenizer1.texts_to_sequences(title) 
padded1 = pad_sequences(sequences1, padding=padding_type, truncating=trunc_type) 
split = int(test_portion * training_size) 
training_sequences1 = padded1[split:training_size] 
test_sequences1 = padded1[0:split] 
test_labels = labels[0:split] 
training_labels = labels[split:training_size] 

In [29]:
embeddings_index = {} 
with open('glove.6B.50d.txt',encoding='UTF-8') as f: 
    for line in f: 
        values = line.split() 
        word = values[0] 
        coefs = np.asarray(values[1:], dtype='float32') 
        embeddings_index[word] = coefs

        
# Generating embeddings 
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim)) 
for word, i in word_index1.items(): 
    embedding_vector = embeddings_index.get(word) 
    if embedding_vector is not None: 
        embeddings_matrix[i] = embedding_vector 

In [30]:
model = tf.keras.Sequential([ 
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim, 
							input_length=max_length, weights=[ 
								embeddings_matrix], 
							trainable=False), 
	tf.keras.layers.Dropout(0.2), 
	tf.keras.layers.Conv1D(64, 5, activation='relu'), 
	tf.keras.layers.MaxPooling1D(pool_size=4), 
	tf.keras.layers.LSTM(64), 
	tf.keras.layers.Dense(1, activation='sigmoid') 
]) 
model.compile(loss='binary_crossentropy', 
			optimizer='adam', metrics=['accuracy']) 
model.summary() 


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_1 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

In [31]:
num_epochs = 50

training_padded = np.array(training_sequences1) 
training_labels = np.array(training_labels) 
testing_padded = np.array(test_sequences1) 
testing_labels = np.array(test_labels) 

history = model.fit(training_padded, training_labels, 
					epochs=num_epochs, 
					validation_data=(testing_padded, 
									testing_labels), 
					verbose=2) 


Train on 2700 samples, validate on 300 samples
Epoch 1/50


C:\Users\Harshal Tak\OneDrive\Desktop\Desktop\TFProjects\tfvenv\lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6384 - accuracy: 0.6230 - val_loss: 0.5561 - val_accuracy: 0.6967 - 3s/epoch - 980us/sample
Epoch 2/50
2700/2700 - 1s - loss: 0.5849 - accuracy: 0.6952 - val_loss: 0.5384 - val_accuracy: 0.7033 - 1s/epoch - 442us/sample
Epoch 3/50
2700/2700 - 1s - loss: 0.5385 - accuracy: 0.7300 - val_loss: 0.5093 - val_accuracy: 0.7333 - 1s/epoch - 439us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.4972 - accuracy: 0.7581 - val_loss: 0.5062 - val_accuracy: 0.7267 - 1s/epoch - 445us/sample
Epoch 5/50
2700/2700 - 1s - loss: 0.4469 - accuracy: 0.7874 - val_loss: 0.5105 - val_accuracy: 0.7633 - 1s/epoch - 449us/sample
Epoch 6/50
2700/2700 - 1s - loss: 0.4021 - accuracy: 0.8230 - val_loss: 0.6209 - val_accuracy: 0.7067 - 1s/epoch - 466us/sample
Epoch 7/50
2700/2700 - 1s - loss: 0.3709 - accuracy: 0.8433 - val_loss: 0.4830 - val_accuracy: 0.7533 - 1s/epoch - 478us/sample
Epoch 8/50
2700/2700 - 1s - loss: 0.3408 - accuracy: 0.8474 - val_loss: 0.5261 - val_accuracy: 0.7533 - 1s/epoch - 

In [32]:
# sample text to check if fake or not 
X = "Karry to go to France in gesture of sympathy"

# detection 
sequences = tokenizer1.texts_to_sequences([X])[0] 
sequences = pad_sequences([sequences], maxlen=54, 
						padding=padding_type, 
						truncating=trunc_type) 
if(model.predict(sequences, verbose=0)[0][0] >= 0.5): 
	print("This news is True") 
else: 
	print("This news is false") 


C:\Users\Harshal Tak\OneDrive\Desktop\Desktop\TFProjects\tfvenv\lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


This news is True
